# 교차 스레드 지속성을 추가하는 방법 (함수형 API)

!!! info "필수 조건"

    이 가이드는 다음에 대한 익숙함을 가정합니다:
    
    - [함수형 API](../../concepts/functional_api/)
    - [지속성](../../concepts/persistence/)
    - [메모리](../../concepts/memory/)
    - [채팅 모델](https://python.langchain.com/docs/concepts/chat_models/)

LangGraph를 사용하면 **다른 [스레드](../../concepts/persistence/#threads)** 간에 데이터를 유지할 수 있습니다. 예를 들어, 사용자에 대한 정보(이름이나 선호도)를 공유(교차 스레드) 메모리에 저장하고 새 스레드(예: 새 대화)에서 재사용할 수 있습니다.

[함수형 API](../../concepts/functional_api/)를 사용할 때 [Store](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore) 인터페이스를 사용하여 메모리를 저장하고 검색하도록 설정할 수 있습니다:

1. `Store`의 인스턴스를 생성합니다

    ```python
    from langgraph.store.memory import InMemoryStore, BaseStore
    
    store = InMemoryStore()
    ```

2. `store` 인스턴스를 `entrypoint()` 데코레이터에 전달하고 함수 시그니처에서 `store` 매개변수를 노출합니다:

    ```python
    from langgraph.func import entrypoint

    @entrypoint(store=store)
    def workflow(inputs: dict, store: BaseStore):
        my_task(inputs).result()
        ...
    ```
    
이 가이드에서는 [Store](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore) 인터페이스를 사용하여 구현된 공유 메모리가 있는 워크플로우를 구성하고 사용하는 방법을 보여줍니다.

!!! note 참고

    이 가이드에서 사용되는 [`Store`](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore) API에 대한 지원은 LangGraph `v0.2.32`에 추가되었습니다.

    이 가이드에서 사용되는 [`Store`](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore) API의 __index__ 및 __query__ 인수에 대한 지원은 LangGraph `v0.2.54`에 추가되었습니다.

!!! tip "참고"

    `StateGraph`에 교차 스레드 지속성을 추가해야 하는 경우 이 [how-to 가이드](../cross-thread-persistence)를 확인하세요.

## 설정

먼저 필요한 패키지를 설치하고 API 키를 설정하겠습니다

In [1]:
%%capture --no-stderr
%pip install -U langchain_anthropic langchain_openai langgraph

In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")
_set_env("OPENAI_API_KEY")

!!! tip "LangGraph 개발을 위한 [LangSmith](https://smith.langchain.com) 설정"

    LangSmith에 가입하여 문제를 빠르게 발견하고 LangGraph 프로젝트의 성능을 개선하세요. LangSmith를 사용하면 LangGraph로 구축한 LLM 앱을 디버그, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있습니다 — 시작하는 방법에 대해 자세히 알아보려면 [여기](https://docs.smith.langchain.com)를 참조하세요

## 예제: 장기 메모리가 있는 간단한 챗봇

### 저장소 정의

이 예제에서는 사용자의 선호도에 대한 정보를 검색할 수 있는 워크플로우를 만듭니다. `InMemoryStore`를 정의하여 이를 수행합니다 - 메모리에 데이터를 저장하고 해당 데이터를 쿼리할 수 있는 객체입니다.

`Store` 인터페이스를 사용하여 객체를 저장할 때 두 가지를 정의합니다:

* 객체의 네임스페이스, 튜플(디렉토리와 유사)
* 객체 키(파일 이름과 유사)

우리 예제에서는 네임스페이스로 `("memories", <user_id>)`를 사용하고 각 새 메모리의 키로 임의의 UUID를 사용합니다.

중요한 것은 사용자를 결정하기 위해 노드 함수의 config 키워드 인수를 통해 `user_id`를 전달한다는 것입니다.

먼저 저장소를 정의해 봅시다!

In [3]:
from langgraph.store.memory import InMemoryStore
from langchain_openai import OpenAIEmbeddings

in_memory_store = InMemoryStore(
    index={
        "embed": OpenAIEmbeddings(model="text-embedding-3-small"),
        "dims": 1536,
    }
)

### 워크플로우 생성

In [ ]:
import uuid

from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import BaseMessage
from langgraph.func import entrypoint, task
from langgraph.graph import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.base import BaseStore


model = ChatAnthropic(model="claude-3-5-sonnet-latest")


@task
def call_model(messages: list[BaseMessage], memory_store: BaseStore, user_id: str):
    namespace = ("memories", user_id)
    last_message = messages[-1]
    memories = memory_store.search(namespace, query=str(last_message.content))
    info = "\n".join([d.value["data"] for d in memories])
    system_msg = f"You are a helpful assistant talking to the user. User info: {info}"

    # 사용자가 모델에게 기억하라고 요청하면 새 메모리를 저장합니다
    if "remember" in last_message.content.lower():
        memory = "User name is Bob"
        memory_store.put(namespace, str(uuid.uuid4()), {"data": memory})

    response = model.invoke([{"role": "system", "content": system_msg}] + messages)
    return response


# 참고: entrypoint()를 통해 워크플로우를 생성할 때 여기에 store 객체를 전달합니다
@entrypoint(checkpointer=InMemorySaver(), store=in_memory_store)
def workflow(
    inputs: list[BaseMessage],
    *,
    previous: list[BaseMessage],
    config: RunnableConfig,
    store: BaseStore,
):
    user_id = config["configurable"]["user_id"]
    previous = previous or []
    inputs = add_messages(previous, inputs)
    response = call_model(inputs, store, user_id).result()
    return entrypoint.final(value=response, save=add_messages(inputs, response))

!!! note 참고

    LangGraph Cloud 또는 LangGraph Studio를 사용하는 경우 자동으로 수행되므로 entrypoint 데코레이터에 store를 전달할 __필요가 없습니다__.

### 워크플로우 실행!

이제 config에서 사용자 ID를 지정하고 모델에 이름을 알려주겠습니다:

In [5]:
config = {"configurable": {"thread_id": "1", "user_id": "1"}}
input_message = {"role": "user", "content": "Hi! Remember: my name is Bob"}
for chunk in workflow.stream([input_message], config, stream_mode="values"):
    chunk.pretty_print()

================================== Ai Message ==================================

Hello Bob! Nice to meet you. I'll remember that your name is Bob. How can I help you today?


In [6]:
config = {"configurable": {"thread_id": "2", "user_id": "1"}}
input_message = {"role": "user", "content": "what is my name?"}
for chunk in workflow.stream([input_message], config, stream_mode="values"):
    chunk.pretty_print()

================================== Ai Message ==================================

Your name is Bob.


이제 인메모리 저장소를 검사하여 사용자에 대한 메모리를 실제로 저장했는지 확인할 수 있습니다:

In [7]:
for memory in in_memory_store.search(("memories", "1")):
    print(memory.value)

{'data': 'User name is Bob'}


이제 다른 사용자에 대해 워크플로우를 실행하여 첫 번째 사용자에 대한 메모리가 자체 포함되어 있는지 확인해 봅시다:

In [8]:
config = {"configurable": {"thread_id": "3", "user_id": "2"}}
input_message = {"role": "user", "content": "what is my name?"}
for chunk in workflow.stream([input_message], config, stream_mode="values"):
    chunk.pretty_print()

================================== Ai Message ==================================

I don't have any information about your name. I can only see our current conversation without any prior context or personal details about you. If you'd like me to know your name, feel free to tell me!
